# PubMed Analysis Statistics

This notebook loads the `pubmed_analysis.parquet` file and calculates comprehensive statistics for all processed columns including:

- Publication metadata (dates, journals, authors, etc.)
- Swiss affiliation analysis
- Goldhamster model predictions
- Text analysis metrics
- Data quality assessment

## 1. Import Required Libraries

Import all necessary libraries for data analysis and statistics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Analysis timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 1.26.4
Analysis timestamp: 2025-12-09 11:49:05


## 2. Load the Parquet File

Load the processed PubMed analysis data from the parquet file.

In [2]:
# Define file path
parquet_file = Path("../data/results/pubmed_analysis.parquet")
    
# Load the data
df = pd.read_parquet(parquet_file)

print(f"✅ Successfully loaded {len(df):,} records")
print(f"File size: {parquet_file.stat().st_size / 1024 / 1024:.1f} MB")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
    
df['mesh_terms'] = df['mesh_terms'].str[2:-2].str.split("', '")

✅ Successfully loaded 350,291 records
File size: 585.6 MB
Memory usage: 2251.9 MB
Memory usage: 2251.9 MB


In [3]:
df.head()

,pmid,title,abstract,publication_date,journal_title,journal_iso_abbreviation,journal_issn,doi,mesh_terms,keywords,authors,download_timestamp,any_author_has_swiss_affiliation,first_author_has_swiss_affiliation,last_author_has_swiss_affiliation,author_count,mesh_term_count,keyword_count,has_abstract,publication_year,publication_month,goldhamster_in_silico,goldhamster_organs,goldhamster_other,goldhamster_human,goldhamster_in_vivo,goldhamster_invertebrate,goldhamster_primary_cells,goldhamster_immortal_cell_line,processing_timestamp
0,24945814,Introgression from domestic goat generated var...,The major histocompatibility complex (MHC) is ...,2014-06-01,PLoS genetics,PLoS Genet,1553-7404,10.1371/journal.pgen.1004438,"[Animals, Base Sequence, Goats, HLA-DR beta-Ch...",[],"[{'last_name': 'Grossen', 'first_name': 'Chris...",2025-12-06T10:12:00.664494,True,True,False,5,10,0,True,2014.0,6.0,0,0,0,0,1,0,0,0,2025-12-09T11:13:20.440868
1,24895028,Divergent dynamics and the Kauzmann temperatur...,In the last decade the challenging analysis of...,2014-06-04,Scientific reports,Sci Rep,2045-2322,10.1038/srep05160,[],[],"[{'last_name': 'Martinez-Garcia', 'first_name'...",2025-12-06T10:12:02.617077,True,True,False,5,0,0,True,2014.0,6.0,0,0,1,0,0,0,0,0,2025-12-09T11:13:20.440868
2,25053935,Physiology of iron metabolism.,A revolution occurred during the last decade i...,2014-06-01,Transfusion medicine and hemotherapy : offizie...,Transfus Med Hemother,1660-3796,10.1159/000362888,[],"['Iron', 'Metabolism', 'Transfusion medicine']","[{'last_name': 'Waldvogel-Abramowski', 'first_...",2025-12-06T10:11:53.302400,True,True,True,7,0,3,True,2014.0,6.0,0,0,0,0,0,0,0,0,2025-12-09T11:13:20.440868
3,24837263,Afterload mismatch after MitraClip insertion f...,"Afterload mismatch, defined as acute impairmen...",2014-06-01,The American journal of cardiology,Am J Cardiol,1879-1913,10.1016/j.amjcard.2014.03.015,"[Aged, Echocardiography, Female, Follow-Up Stu...",[],"[{'last_name': 'Melisurgo', 'first_name': 'Giu...",2025-12-06T10:11:29.682134,True,False,True,12,18,0,True,2014.0,6.0,0,0,1,0,0,0,0,0,2025-12-09T11:13:20.440868
4,24656396,CTLA4 polymorphisms in minimal change nephroti...,N/A,2014-06-01,American journal of kidney diseases : the offi...,Am J Kidney Dis,1523-6838,10.1053/j.ajkd.2014.01.427,"[Adolescent, CTLA-4 Antigen, Case-Control Stud...",[],"[{'last_name': 'Ohl', 'first_name': 'Kim', 'in...",2025-12-06T10:11:31.898864,True,False,False,11,13,0,True,2014.0,6.0,0,0,1,0,0,0,0,0,2025-12-09T11:13:20.440868


## 3. Counts by publication_year

In [11]:
df2 = df[[
    'publication_year', 'any_author_has_swiss_affiliation', 
    'first_author_has_swiss_affiliation', 'last_author_has_swiss_affiliation',
    'goldhamster_in_silico', 'goldhamster_organs', 'goldhamster_other', 
    'goldhamster_human', 'goldhamster_in_vivo', 'goldhamster_invertebrate', 
    'goldhamster_primary_cells', 'goldhamster_immortal_cell_line'
]].rename(columns={
    'mesh_terms': 'mesh_term',
    'any_author_has_swiss_affiliation': 'any_auth',
    'first_author_has_swiss_affiliation': 'first_auth',
    'last_author_has_swiss_affiliation': 'last_auth',
    'goldhamster_in_silico': 'in_silico',
    'goldhamster_organs': 'organs',
    'goldhamster_other': 'other',
    'goldhamster_human': 'human',
    'goldhamster_in_vivo': 'in_vivo',
    'goldhamster_invertebrate': 'invertebrate',
    'goldhamster_primary_cells': 'primary_cells',
    'goldhamster_immortal_cell_line': 'immortal_cell_line'
}).groupby('publication_year').sum().reset_index()

df2['publication_year'] = df2['publication_year'].astype('Int64')

df2.to_excel("../data/results/pubmed_analysis_statistics.xlsx", index=False)

df2


,publication_year,any_auth,first_auth,last_auth,in_silico,organs,other,human,in_vivo,invertebrate,primary_cells,immortal_cell_line
0,2009,1,1,0,0,0,1,0,0,0,0,0
1,2010,7889,7816,648,744,148,4213,1249,1089,143,68,78
2,2011,7401,7309,530,697,137,4067,1122,939,136,61,81
3,2012,9598,9417,839,979,181,5223,1385,1256,166,77,67
4,2013,10843,10256,1336,1084,185,6119,1533,1376,178,67,105
5,2014,15913,11285,6673,1600,232,8920,2357,2071,244,106,138
6,2015,18979,12241,9514,1867,275,11112,2761,2339,246,112,130
7,2016,21583,12820,11820,2030,294,12814,3036,2589,317,115,147
8,2017,23260,13372,12336,2297,309,14326,3232,2691,282,122,149
9,2018,23794,13106,12724,2290,321,15089,3155,2621,283,104,135


## 4. Counts by mesh_term

In [10]:
df2 = df[[
    'mesh_terms', 'any_author_has_swiss_affiliation', 
    'first_author_has_swiss_affiliation', 'last_author_has_swiss_affiliation',
    'goldhamster_in_silico', 'goldhamster_organs', 'goldhamster_other', 
    'goldhamster_human', 'goldhamster_in_vivo', 'goldhamster_invertebrate', 
    'goldhamster_primary_cells', 'goldhamster_immortal_cell_line'
]].rename(columns={
    'mesh_terms': 'mesh_term',
    'any_author_has_swiss_affiliation': 'any_auth',
    'first_author_has_swiss_affiliation': 'first_auth',
    'last_author_has_swiss_affiliation': 'last_auth',
    'goldhamster_in_silico': 'in_silico',
    'goldhamster_organs': 'organs',
    'goldhamster_other': 'other',
    'goldhamster_human': 'human',
    'goldhamster_in_vivo': 'in_vivo',
    'goldhamster_invertebrate': 'invertebrate',
    'goldhamster_primary_cells': 'primary_cells',
    'goldhamster_immortal_cell_line': 'immortal_cell_line'
}).explode('mesh_term').groupby(['mesh_term']).sum().reset_index()

df2.to_excel(Path("../data/results/pubmed_mesh_term_statistics.xlsx"), index=False)
print("✅ Exported mesh term statistics to Excel file")

df2

✅ Exported mesh term statistics to Excel file


,mesh_term,any_auth,first_auth,last_auth,in_silico,organs,other,human,in_vivo,invertebrate,primary_cells,immortal_cell_line
0,,91145,52176,48131,13106,851,61097,7558,5673,499,182,213
1,"1,2-Dipalmitoylphosphatidylcholine",19,9,7,5,4,1,1,2,0,0,0
2,"1,4-alpha-Glucan Branching Enzyme",4,3,2,0,0,1,0,1,0,0,0
3,1-(5-Isoquinolinesulfonyl)-2-Methylpiperazine,5,1,1,0,0,1,1,2,0,0,0
4,1-Acylglycerol-3-Phosphate O-Acyltransferase,3,2,2,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
26761,"vif Gene Products, Human Immunodeficiency Virus",1,0,0,0,0,1,0,0,0,0,0
26762,von Hippel-Lindau Disease,23,14,6,0,0,16,8,1,0,0,0
26763,von Willebrand Diseases,22,12,7,0,0,16,6,1,0,0,0
26764,von Willebrand Factor,94,55,29,0,4,36,36,14,0,2,0
